In [1]:
import pandas as pd
from pathlib import Path

# === Config ===
INPUT_FILE = "My EndNote Library.xlsx"
OUTPUT_XLSX = "My EndNote Library Dedup.xlsx"

# --- Load ---
path = Path(INPUT_FILE)
if not path.exists():
    raise FileNotFoundError(f"Could not find {INPUT_FILE} in the current directory.")

df = pd.read_excel(path)

if "Title" not in df.columns:
    raise KeyError("The input file must contain a 'Title' column.")

# --- Normalize Title for case/space-insensitive duplicate detection ---
# (Preserves NaN — NaNs are NOT treated as duplicates)
df["_Title_norm"] = (
    df["Title"]
      .str.strip()
      .str.lower()
      .str.replace(r"\s+", " ", regex=True)
)

# --- Deduplicate: keep first occurrence of each normalized title ---
records_identified = len(df)
df_clean = df.drop_duplicates(subset=["_Title_norm"], keep="first").drop(columns=["_Title_norm"])
records_after_dedup = len(df_clean)
duplicates_removed = records_identified - records_after_dedup

# --- Save cleaned Excel file ---
df_clean.to_excel(OUTPUT_XLSX, index=False)

# --- Print concise PRISMA-style summary ---
print("=== PRISMA-style Deduplication Summary ===")
print(f"Records identified (imported): {records_identified}")
print("Duplicates criterion: Title (case- & space-insensitive)")
print(f"Duplicates removed: {duplicates_removed}")
print(f"Records after duplicates removed: {records_after_dedup}")
print(f"\nCleaned dataset saved to: {OUTPUT_XLSX}")

=== PRISMA-style Deduplication Summary ===
Records identified (imported): 2363
Duplicates criterion: Title (case- & space-insensitive)
Duplicates removed: 307
Records after duplicates removed: 2056

Cleaned dataset saved to: My EndNote Library Dedup.xlsx
